# Weather Data Analysis

### Import Dependencies

In [ ]:
#import dependencies

import random
import requests
import pandas as pd
import csv
import matplotlib.pyplot as plt

from citipy import citipy
from config import api_key


### Random latitudes

- Range Latitudes: -90 to +90
- Range Longitudes: -180 to +180

In [ ]:
coord_list = []

#Generate 600 random lats and lons

ix = 0
#jx = 150
jx = 10

while(ix<18):
    
    range1 = -90 + (ix*10)
    range2 = range1 + 10 
    
    range3 = -180 + (ix*20)
    range4 = range3 + 20 
    
    print(f'#### {ix}) Get {jx} random Lats from {range1}, {range2} and random Long from {range3}, {range4}')
    
    for i in range(jx):
        
        rand_lat = round( random.uniform(range1, range2) , 5) 
        rand_lon = round( random.uniform(range3, range4) , 5) 

        tmp = (rand_lat,rand_lon)
        coord_list.append(tmp)
    
    ix += 1
    
## I had issues finding UNIQUE cities because the rando was too wide.. 
## so to solve this I started getting data from ranges from 10 to 10.. this is getting random numbers 
# from -90 to -80, then from -80 to -70

In [ ]:
# Example
i = 0
for coord in coord_list:
    print(coord)
    
    if i>3:
        break
    i += 1
    

### Get the cities

These have to be at least 500 unique cities

In [ ]:
cities = []

for coord in coord_list:
    
    city = citipy.nearest_city(coord[0],coord[1])
    city_name = city.city_name
    country_code = city.country_code
    #print(f'processing #  City Name: {city_name} - Country Code: {country_code}')
    
    tmp = (city_name, country_code)
    cities.append(tmp)

In [ ]:
len(cities)

In [ ]:
unique_cities = list(set(cities))

In [ ]:
len(unique_cities)

#### OpenWeatherMap - Weather API

In [ ]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
query_url_tmp = f'{url}appid={api_key}&units={units}&'


In [ ]:
#coord_list = [(lat,lon)]
city_name_list = []
country_code_lsit = []

lon_api_list = []
lat_api_list = []
temp_list = []
humidity_list = []
wind_list = []
clouds = []
dt = []
country_api = []
city_id_api = []
city_api = []

i = 0
j = 0
# Loop through the list of cities and perform a request for data on each
for cities in unique_cities:
    
    print(f'{j}-{i}: processing #  City Name: {cities[0]} - Country Code: {cities[1]}')
    
    query_url = query_url_tmp + f'q={cities[0]},{cities[1]}'
    
    #print(query_url) #do not print when pushing to github
    try:
        response = requests.get(query_url).json()
        response['sys']['country']
        j += 1
    except:
        print(f'#### Not Found: {cities[0]} in {cities[1]}')
        continue
    
    city_name_list.append(cities[0])
    country_code_lsit.append(cities[1])
    
    lon_api_list.append(response['coord']['lon'])
    lat_api_list.append(response['coord']['lat'])
    temp_list.append(response['main']['temp'])
    humidity_list.append(response['main']['humidity'])
    wind_list.append(response['wind']['speed'])
    clouds.append(response['clouds']['all'])
    dt.append(response['dt'])
    country_api.append(response['sys']['country'])
    city_id_api.append(response['id'])
    city_api.append(response['name'])
    
    #print(response)
    i += 1


In [ ]:
weather_data = pd.DataFrame(
    {'city_name_list': city_name_list,
    'country_code_lsit': country_code_lsit,
    'lon_api_list': lon_api_list,
    'lat_api_list': lat_api_list,
    'temp_list': temp_list,
    'humidity_list': humidity_list,
    'wind_list': wind_list,
    'clouds': clouds,
     'city_id_api': city_id_api,
     'city_api': city_api,
    'country_api': country_api,
     'dt': dt
    })


weather_data

In [ ]:
coord_list[16]

In [ ]:
print(len(city_name_list))
print(len(country_code_lsit))

print(len(lon_api_list))
print(len(lat_api_list))
print(len(temp_list))
print(len(humidity_list))
print(len(wind_list))
print(len(clouds))
print(len(dt))
print(len(country_api))
print(len(city_id_api))
print(len(city_api))




---

### Data analysis

- Temperature (F) vs. Latitude
- Humidity (%) vs. Latitude
- Cloudiness (%) vs. Latitude
- Wind Speed (mph) vs. Latitude